<a href="https://colab.research.google.com/github/Tagore-7/python_projects/blob/main/advanced_python_scrpting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### get_game_data.py

import os
import json
import shutil
from subprocess import PIPE, run
import sys


GAME_DIR_PATTERM = "game"
GAME_CODE_EXTENSION = ".go"
GAME_COMPILE_COMMAND = ["go", "build"]

def find_all_game_paths(source):
  game_path = []

  for root, dirs, files in os.walk(source):
    for directory in dirs:
      if GAME_DIR_PATTERM in dirs:
        path = os.path.join(source, directory)
        game_paths.append(path)
    break

def get_name_from_path(paths, to_strip):
  new_names = []
  for path in paths:
    _, dir_name = os.path.split(path)
    new_dir_name = dir_name.replace(to_strip,"")
    new_names.append(new_dir_name)

  return new_names


def create_dir(path):
  if not os.path.exists(path):
    os.mkdir(path)

def copy_and_overwrite(source, dest):
  if os.path.exists(dest):
    shutil.rmtree(dest)
  shutil.copytree(source, dest)


def make_json_metadata_file(path, game_dirs):
  data = {
      "gameNames" : game_dirs,
      "numberOfGames": len(game_dirs)
  }
  with open(path, "w") as f:
    json.dump(data, f)


def compile_game_code(path):
  code_file_name = None
  for root, dirs, files in os.walk(path):
    for file in files:
      if  file.endswith(GAME_CODE_EXTENSION):
        code_file_name = file
        break
    break


  if code_file_name is None:
    return

  command = GAME_COMPILE_COMMAND + [code_file_name]

def run_command(command, path):
  cwd = os.getcwd()
  os.chdir(path)
  result = run(command, stdout=PIPE, stdin=PIPE, universal_newlines=True)
  print(result)

  os.chdir(cwd)


def main(source, target):
  cwd = os.getcwd()
  source_path = os.path.join(cwd, source)
  target_path = os.path.join(cwd, target)

  game_paths = find_all_game_paths(source_path)
  # print(game_paths)
  new_game_dirs = get_name_from_path(game_paths, "_games")
  # print(new_game_dirs)

  create_dir(target_path)

  for src, dest in zip(game_paths, new_game_dirs):
    dest_path = os.path.join(target_path, dest)
    copy_and_overwrite(src, dest_path)
    compile_game_code(dest_path)

  json_path = os.path.join(target_path, "metadata.json")
  make_json_metadata_file(json_path, new_game_dirs)



if __name__ == '__main__':
  args = sys.argv
  # print(args)
  if len(args) != 3:
    raise Exception("You must pass a source and target director- only")

  source, target = args[1:]
  main(source, target)


